In [2]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')


In [3]:
# read data and save in matrix form
threshold_mode = 'count'
num_nn = 4
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'

colony0_features = pd.read_csv(os.path.join(
    path_to_data, 'colony0_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]
colony0_matrix_features = get_matrix_features(
    colony0_features, colony0_lineage_gt)

# other colonies
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)

colonies_features = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    colonies_features = pd.concat([pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_candidate_features_{}_{}_nn.csv'.format(i, threshold_mode, num_nn))), colonies_features], ignore_index=True)

colonies_matrix_features = get_matrix_features(colonies_features, colonies_gt)


parent not in candidates 168 0 [ 33  77 159 154] 1
parent not in candidates 280 0 [105 212  27 238] 11
parent not in candidates 358 0 [-3 -3 -3 -3] 171
parent not in candidates 359 0 [-3 -3 -3 -3] 203
parent not in candidates 360 0 [-3 -3 -3 -3] 86
parent not in candidates 361 0 [-3 -3 -3 -3] 176
parent not in candidates 362 0 [-3 -3 -3 -3] 81
parent not in candidates 363 0 [-3 -3 -3 -3] 177
parent not in candidates 364 0 [-3 -3 -3 -3] 91
parent not in candidates 365 0 [-3 -3 -3 -3] 132
parent not in candidates 366 0 [-3 -3 -3 -3] 111
parent not in candidates 367 0 [-3 -3 -3 -3] 224
parent not in candidates 368 0 [-3 -3 -3 -3] 174
parent not in candidates 153 2 [-3 -3 -3 -3] 118
parent not in candidates 154 2 [-3 -3 -3 -3] 108
parent not in candidates 155 2 [-3 -3 -3 -3] 53
parent not in candidates 96 3 [-3 -3 -3 -3] 27
parent not in candidates 97 3 [-3 -3 -3 -3] 8
parent not in candidates 40 4 [-3 -3 -3 -3] 24
parent not in candidates 105 4 [-3 -3 -3 -3] 67
parent not in candidates 13

In [4]:
# read in data for colonies 1 to 5 and colony 0 and combine their matrices

# combine all data to make a single dataframe for all colonies
all_matrix_features = pd.concat(
    [colony0_matrix_features, colonies_matrix_features]).reset_index(drop=True)
all_matrix_features


,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
0,1,5,18,0,"[[2.23606797749979, 2.8284271247461903, 2.3462...","[1, 3, 2, 4]",0
1,3,6,18,0,"[[1.0, 3.0, 3.111078760931904, 3.0597171328670...","[3, 4, 2, 1]",0
2,2,7,20,0,"[[1.0, 3.0, 0.9423480319763352, 1.007736816321...","[2, 3, 4, 6]",0
3,4,8,22,0,"[[1.0, 2.0, 2.107542585550348, 2.1228587110116...","[4, 7, 6, 2]",0
4,1,9,32,0,"[[2.0, 2.0, 1.7589301904710015, 1.703711558127...","[1, 5, 3, 2]",0
...,...,...,...,...,...,...,...
821,22,34,164,5,"[[2.0, 3.0, 1.1800917718383046, 1.191595609891...","[22, 25, 8, 11]",0
822,13,35,166,5,"[[2.0, 3.0, 0.8067932399673147, 0.729620098766...","[13, 16, 10, 4]",0
823,18,36,169,5,"[[2.23606797749979, 4.123105625617661, 1.47672...","[18, 9, 2, 32]",0
824,3,37,175,5,"[[2.0, 2.23606797749979, 1.591168586220782, 1....","[18, 26, 3, 27]",2


In [5]:
# define a module to perform cross-validation for XGBoost
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from utils import generate_all_permutations, flatten_3d_array, grid_train_xgboost


def run_xgboost(X_train, y_train, X_test, y_test, params={}, augment=True):
    if augment:
        X_train, y_train = generate_all_permutations(X_train, y_train)
    X_train = flatten_3d_array(X_train)
    X_test = flatten_3d_array(X_test)

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    if params == {}:  # if no parameters are provided, use the best parameters found from grid search
        params = grid_train_xgboost(
            X_train, y_train, X_test, y_test, early_stopping_rounds=5, num_boost_round=200)
        print('best parameters found from grid search:', params)

    # Initialize XGBoost model and fit to training data
    model = xgb.train(params, dtrain, evals=[(dtest, 'test')],
                      num_boost_round=200, verbose_eval=False, early_stopping_rounds=5)
    preds = model.predict(dtest)
    preds = np.round(preds)
    print("Accuracy of the best model:", "%.4f " %
          accuracy_score(y_test, preds))
    return model, preds


def cv_xgboost(X, y, augment=True):
    # Initialize cross-validator
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Train and evaluate model for each fold
    fold_scores = []
    models = []
    for train_idx, test_idx in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]

        model, preds = run_xgboost(
            X_train, y_train, X_test, y_test, augment=augment)

        # Compute predictions on test data and evaluate accuracy
        fold_score = accuracy_score(y_test, preds)
        fold_scores.append(fold_score)
        models.append(model)
    # Compute mean and standard deviation of fold scores
    mean_score = np.mean(fold_scores)
    std_score = np.std(fold_scores)

    # Print validation results
    print('Mean Score: {:.3f} +/- {:.3f}'.format(mean_score, std_score))
    return mean_score, std_score, models


In [6]:
# train and test for all colonies
mean_score, std_score, models_all = cv_xgboost(all_matrix_features['features'].to_numpy(),
                                               all_matrix_features['parent_index_in_candidates'].to_numpy())


best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 10, 'min_child_weight': 10, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8434 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 5, 'min_child_weight': 10, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8545 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 10, 'min_child_weight': 1, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8424 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 5, 'min_child_weight': 2, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8424 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 5, 'min_child_weight': 2, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8545 
Mean Score: 0.847 +/- 0.006


## test the above XGBoost on the external data


In [9]:
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_features_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/features/features.csv'
threshold_mode = 'count'
num_nn = 4
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'

external_subset_features = pd.read_csv(os.path.join(
    path_to_features, 'external_subset_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))
external_subset_lineage_gt= pd.read_csv(external_subset_lineage_GT_path_edited)

# create matrix features for the external subset
external_subset_matrix_features = get_matrix_features(
    external_subset_features, external_subset_lineage_gt)

# test for the external subset
X = external_subset_matrix_features['features'].to_numpy()
y = external_subset_matrix_features['parent_index_in_candidates'].to_numpy()
accuracies = []
for model in models_all:
    preds = model.predict(xgb.DMatrix(flatten_3d_array(X)))
    accuracy = accuracy_score(y, np.round(preds))
    accuracies.append(accuracy)
print('accuracy for external data subset:', np.mean(accuracies), '+/-', np.std(accuracies))


parent not in candidates 287 10 [269 207 122  92] -3
parent not in candidates 289 10 [269 207 197  61] -3
parent not in candidates 291 10 [269 207  92 122] -3
parent not in candidates 311 10 [-3 -3 -3 -3] 16
parent not in candidates 312 10 [-3 -3 -3 -3] 15
parent not in candidates 313 10 [-3 -3 -3 -3] 97
parent not in candidates 314 10 [-3 -3 -3 -3] 83
parent not in candidates 315 10 [-3 -3 -3 -3] 7
parent not in candidates 316 10 [-3 -3 -3 -3] 38
parent not in candidates 317 10 [-3 -3 -3 -3] 130
parent not in candidates 318 10 [-3 -3 -3 -3] 159
parent not in candidates 319 10 [-3 -3 -3 -3] 77
parent not in candidates 320 10 [-3 -3 -3 -3] 144
parent not in candidates 322 10 [-3 -3 -3 -3] 209
parent not in candidates 323 10 [-3 -3 -3 -3] 289
parent not in candidates 324 10 [-3 -3 -3 -3] 75
parent not in candidates 325 10 [-3 -3 -3 -3] 58
parent not in candidates 326 10 [-3 -3 -3 -3] 310
parent not in candidates 327 10 [-3 -3 -3 -3] 145
parent not in candidates 328 10 [-3 -3 -3 -3] 287
p

## Train XGBoost on 5 colonies and test on colony 0


In [8]:
# train and test for colony 1 to 5
mean_score, std_score, models = cv_xgboost(colonies_matrix_features['features'].to_numpy(),
                                           colonies_matrix_features['parent_index_in_candidates'].to_numpy())


/home/farzaneh/anaconda3/envs/lineage_tracing/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 10, 'min_child_weight': 5, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8542 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 5, 'min_child_weight': 5, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8438 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 5, 'min_child_weight': 2, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8632 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 10, 'min_child_weight': 10, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8105 
best parameters found from grid search: {'objective': 'multi:softmax', 'num_class': 4, 'max_depth': 10, 'min_child_weight': 1, 'eval_metric': 'merror'}
Accuracy of the best model: 0.8526 
Mean Score: 0.845 +/- 0.018


In [7]:
X = colony0_matrix_features['features'].to_numpy()
y = colony0_matrix_features['parent_index_in_candidates'].to_numpy()
accuracies = []
for model in models:
    preds = model.predict(xgb.DMatrix(flatten_3d_array(X)))
    accuracy = accuracy_score(y, np.round(preds))
    accuracies.append(accuracy)
print('accuracy for colony 0:', np.mean(accuracies), '+/-', np.std(accuracies))


NameError: name 'models' is not defined